## Создание ассистентов

В этом ноутбуке мы попробуем создать и протестировать чат-ассистента на основе Yandex Assistant API, RAG и Function Calling. Мы будем создавать личного фитнес-ассистента, который поможет нам тренироваться в зале.

Для начала, установим Yandex Cloud ML SDK:

In [ ]:
%pip install --upgrade --quiet yandex-cloud-ml-sdk==0.13.1 pydantic==2.11.4

**ВНИМАНИЕ**: После установки библиотек рекомендуется перезапустить Kernel ноутбука.

И ещё пара полезных функций на будущее:

In [52]:
from IPython.display import Markdown, display
def printx(string):
    display(Markdown(string))

Для работы с языковыми моделями нам понадобится ключ `api_key` для сервисного аккаунта, имеющего права на доступ к модели, и `folder_id`. Мы предполагаем, что соответствующие значения хранятся в секретах Datasphere, или установлены в вашей переменной окружения.

Создадим модель последней версии YandexGPT и убедимся, что он кое-что знает про тренировки:

In [53]:
import os
from yandex_cloud_ml_sdk import YCloudML

folder_id = os.environ["folder_id"]
api_key = os.environ["api_key"]

sdk = YCloudML(folder_id=folder_id, auth=api_key)

# Раскомментируйте, если хотите подробнее смотреть, что делает SDK
#sdk.setup_default_logging(log_level='DEBUG')

model = sdk.models.completions("yandexgpt", model_version="rc")

In [54]:
printx(model.run("Как тренироваться, чтобы сбросить вес?").text)

Перед началом любой программы тренировок для снижения веса рекомендуется проконсультироваться с врачом или квалифицированным тренером, чтобы убедиться, что выбранный режим физической активности подходит вам и не навредит здоровью.

Вот несколько общих рекомендаций по тренировкам для снижения веса:

1. **Комбинируйте кардио и силовые тренировки:** кардиоупражнения помогают сжигать калории и улучшать сердечно-сосудистую систему, а силовые тренировки способствуют укреплению мышц и ускорению обмена веществ.

2. **Начните с умеренной интенсивности:** если вы новичок в тренировках, начните с умеренной интенсивности и постепенно увеличивайте нагрузку. Это поможет избежать травм и перетренированности.

3. **Разнообразьте тренировки:** чтобы не допустить привыкания организма к однотипным нагрузкам, разнообразьте свои тренировки. Включайте в них разные виды кардио (бег, плавание, велосипедные прогулки) и силовые упражнения (работа с собственным весом, гантелями, тренажёрами).

4. **Следите за временем и частотой тренировок:** для достижения результатов важно не только качество тренировок, но и их продолжительность и частота. Рекомендуется заниматься не менее 3–4 раз в неделю по 30–60 минут.

5. **Не забывайте о разминке и заминке:** перед каждой тренировкой проводите разминку, чтобы подготовить организм к нагрузке, а после тренировки — заминку для постепенного снижения пульса и расслабления мышц.

6. **Соблюдайте режим питания:** тренировки будут более эффективными, если вы следите за своим рационом. Старайтесь употреблять больше овощей, фруктов, белков и сложных углеводов, а также контролировать калорийность питания.

7. **Будьте терпеливы:** снижение веса — это процесс, который требует времени и усилий. Не ожидайте быстрых результатов и будьте готовы к тому, что прогресс может быть нелинейным.

8. **Следите за самочувствием:** если во время тренировок вы чувствуете боль, дискомфорт или другие неприятные симптомы, прекратите тренировку и обратитесь к врачу.

9. **Записывайте свои результаты:** ведите дневник тренировок, где будете записывать свои достижения, время тренировок, интенсивность и другие параметры. Это поможет вам отслеживать прогресс и корректировать программу тренировок при необходимости.

При вызове модели можно передавать также историю переписки, включая системны промпт:

In [55]:
model.run([
    { "role": "system", "text": "Ты - опытный фитнес-тренер, задача которого - помочь мне тренироваться в зале." },
    { "role": "user", "text": "Привет! С чего ты порекомендуешь начать тренировки в зале?" },
])

GPTModelResult(alternatives=(Alternative(role='assistant', text='Привет! Прежде чем начать тренировки, важно учесть несколько моментов:\n\n1. **Консультация с врачом.** Если у вас есть какие-либо хронические заболевания или ограничения по здоровью, обязательно проконсультируйтесь с врачом перед началом тренировок.\n\n2. **Определение целей.** Подумайте, чего вы хотите достичь с помощью тренировок: похудеть, набрать мышечную массу, улучшить общую физическую форму и т. д. Это поможет вам выбрать правильный план тренировок и диету.\n\n3. **Подбор экипировки.** Убедитесь, что у вас есть удобная спортивная одежда и обувь, которые не будут стеснять движений и вызывать дискомфорт во время тренировок.\n\n4. **Знакомство с залом.** Перед началом тренировок ознакомьтесь с оборудованием и правилами зала. Это поможет вам чувствовать себя более уверенно и избежать травм.\n\n5. **Разминка.** Перед каждой тренировкой обязательно проводите разминку, чтобы подготовить мышцы и суставы к нагрузке. Размин

## Assistant API

Для ведения беседы с моделью с сохранением контекста диалога используем Assistants API. Объект `thread` будет отвечать за сохранение контекста, а `assistant` - за все основные установки, связанные с работой ассистента.

In [56]:
def create_thread():
    return sdk.threads.create(ttl_days=1, expiration_policy="static")

def create_assistant(model, tools=None):
    kwargs = {}
    if tools and len(tools) > 0:
        kwargs = {"tools": tools}
    return sdk.assistants.create(
        model, ttl_days=1, expiration_policy="since_last_active", **kwargs
    )

Создадим простого ассистента и беседу:

In [57]:
thread = create_thread()
assistant = create_assistant(model)

assistant.update(
    instruction="Ты - опытный фитнес-тренер, задача которого - помочь мне тренироваться в зале."
)

thread.write("Привет! С чего ты порекомендуешь начать тренировки в зале?")

run = assistant.run(thread)
result = run.wait()

printx(result.text)

Привет! Прежде чем начать тренировки, важно учесть несколько моментов:

1. **Консультация с врачом.** Если у вас есть какие-либо хронические заболевания или ограничения по здоровью, обязательно проконсультируйтесь с врачом перед началом тренировок.

2. **Определение целей.** Подумайте, чего вы хотите достичь с помощью тренировок: похудеть, набрать мышечную массу, улучшить общую физическую форму и т. д. Это поможет вам и вашему тренеру разработать наиболее эффективную программу.

3. **Выбор программы тренировок.** Лучше всего обратиться к профессиональному тренеру, который поможет вам разработать индивидуальную программу, учитывая ваши цели, уровень подготовки и особенности здоровья. Если вы предпочитаете самостоятельные тренировки, то можете изучить различные программы в интернете или книгах, но помните, что индивидуальный подход всегда более эффективен.

4. **Разминка.** Перед каждой тренировкой обязательно проводите разминку, чтобы подготовить мышцы и суставы к нагрузке. Разминка может включать в себя кардиоупражнения (бег на месте, прыжки со скакалкой) и динамическую растяжку.

5. **Техника выполнения упражнений.** Правильная техника выполнения упражнений — это залог эффективности тренировок и предотвращения травм. Если вы новичок, лучше всего обратиться к тренеру, который покажет вам правильную технику и проконтролирует её выполнение.

6. **Постепенное увеличение нагрузки.** Не стоит сразу же пытаться поднять максимальный вес или выполнить максимальное количество повторений. Увеличивайте нагрузку постепенно, давая организму время адаптироваться.

7. **Отдых и восстановление.** Не менее важно давать организму время на восстановление между тренировками. Отдых помогает мышцам восстановиться и расти, а также предотвращает перетренированность.

8. **Питание.** Правильное питание играет важную роль в достижении спортивных целей. Убедитесь, что ваш рацион содержит достаточное количество белков, углеводов и жиров, а также витаминов и минералов.

9. **Гидратация.** Не забывайте пить достаточное количество воды во время тренировок и в течение дня.

10. **Отслеживание прогресса.** Ведите дневник тренировок, записывайте свои результаты и отслеживайте прогресс. Это поможет вам видеть свои достижения и корректировать программу при необходимости.

In [58]:
thread.write("Я хочу похудеть!")

run = assistant.run(thread)
result = run.wait()

printx(result.text)

Хорошо, давайте разработаем план, который поможет вам похудеть. Вот несколько рекомендаций:

1. **Консультация с врачом и тренером.** Прежде чем начать, убедитесь, что ваше здоровье позволяет вам заниматься спортом. Врач даст необходимые рекомендации, а тренер поможет разработать безопасную и эффективную программу тренировок.

2. **Комбинируйте кардио и силовые тренировки.** Кардио упражнения (бег, быстрая ходьба, езда на велосипеде) помогут сжигать калории во время тренировки, а силовые тренировки (работа с гантелями, собственным весом) помогут укрепить мышцы и ускорить метаболизм.

3. **Разнообразие тренировок.** Чтобы избежать привыкания организма к однотипным нагрузкам, включайте в программу разнообразные упражнения. Это поможет поддерживать интерес к тренировкам и эффективнее сжигать калории.

4. **Контроль калорийности рациона.** Для похудения важно создать дефицит калорий, то есть тратить больше калорий, чем вы потребляете. Однако не снижайте калорийность рациона слишком резко, чтобы не навредить здоровью.

5. **Регулярность тренировок.** Старайтесь заниматься не менее 3–4 раз в неделю. Регулярные тренировки помогут поддерживать высокий уровень метаболизма и ускорят процесс похудения.

6. **Отдых и восстановление.** Не забывайте о важности отдыха. Мышцам нужно время для восстановления после тренировок, поэтому не забывайте про дни отдыха.

7. **Отслеживание прогресса.** Ведите дневник тренировок и записывайте свои результаты. Это поможет вам видеть прогресс и мотивировать на дальнейшие достижения.

8. **Поддержка и мотивация.** Найдите единомышленников или присоединитесь к группе по интересам. Поддержка и мотивация от других людей могут стать дополнительным стимулом для достижения целей.

Помните, что похудение — это процесс, который требует времени и терпения. Главное — быть последовательным и не сдаваться при первых трудностях. Удачи вам!

В итоге в переписке `thread` содержится вся история сообщений:

In [59]:
for msg in list(thread)[::-1]:
    printx(f"**{msg.author.role}:** {msg.text}")

**USER:** Привет! С чего ты порекомендуешь начать тренировки в зале?

**ASSISTANT:** Привет! Прежде чем начать тренировки, важно учесть несколько моментов:

1. **Консультация с врачом.** Если у вас есть какие-либо хронические заболевания или ограничения по здоровью, обязательно проконсультируйтесь с врачом перед началом тренировок.

2. **Определение целей.** Подумайте, чего вы хотите достичь с помощью тренировок: похудеть, набрать мышечную массу, улучшить общую физическую форму и т. д. Это поможет вам и вашему тренеру разработать наиболее эффективную программу.

3. **Выбор программы тренировок.** Лучше всего обратиться к профессиональному тренеру, который поможет вам разработать индивидуальную программу, учитывая ваши цели, уровень подготовки и особенности здоровья. Если вы предпочитаете самостоятельные тренировки, то можете изучить различные программы в интернете или книгах, но помните, что индивидуальный подход всегда более эффективен.

4. **Разминка.** Перед каждой тренировкой обязательно проводите разминку, чтобы подготовить мышцы и суставы к нагрузке. Разминка может включать в себя кардиоупражнения (бег на месте, прыжки со скакалкой) и динамическую растяжку.

5. **Техника выполнения упражнений.** Правильная техника выполнения упражнений — это залог эффективности тренировок и предотвращения травм. Если вы новичок, лучше всего обратиться к тренеру, который покажет вам правильную технику и проконтролирует её выполнение.

6. **Постепенное увеличение нагрузки.** Не стоит сразу же пытаться поднять максимальный вес или выполнить максимальное количество повторений. Увеличивайте нагрузку постепенно, давая организму время адаптироваться.

7. **Отдых и восстановление.** Не менее важно давать организму время на восстановление между тренировками. Отдых помогает мышцам восстановиться и расти, а также предотвращает перетренированность.

8. **Питание.** Правильное питание играет важную роль в достижении спортивных целей. Убедитесь, что ваш рацион содержит достаточное количество белков, углеводов и жиров, а также витаминов и минералов.

9. **Гидратация.** Не забывайте пить достаточное количество воды во время тренировок и в течение дня.

10. **Отслеживание прогресса.** Ведите дневник тренировок, записывайте свои результаты и отслеживайте прогресс. Это поможет вам видеть свои достижения и корректировать программу при необходимости.

**USER:** Я хочу похудеть!

**ASSISTANT:** Хорошо, давайте разработаем план, который поможет вам похудеть. Вот несколько рекомендаций:

1. **Консультация с врачом и тренером.** Прежде чем начать, убедитесь, что ваше здоровье позволяет вам заниматься спортом. Врач даст необходимые рекомендации, а тренер поможет разработать безопасную и эффективную программу тренировок.

2. **Комбинируйте кардио и силовые тренировки.** Кардио упражнения (бег, быстрая ходьба, езда на велосипеде) помогут сжигать калории во время тренировки, а силовые тренировки (работа с гантелями, собственным весом) помогут укрепить мышцы и ускорить метаболизм.

3. **Разнообразие тренировок.** Чтобы избежать привыкания организма к однотипным нагрузкам, включайте в программу разнообразные упражнения. Это поможет поддерживать интерес к тренировкам и эффективнее сжигать калории.

4. **Контроль калорийности рациона.** Для похудения важно создать дефицит калорий, то есть тратить больше калорий, чем вы потребляете. Однако не снижайте калорийность рациона слишком резко, чтобы не навредить здоровью.

5. **Регулярность тренировок.** Старайтесь заниматься не менее 3–4 раз в неделю. Регулярные тренировки помогут поддерживать высокий уровень метаболизма и ускорят процесс похудения.

6. **Отдых и восстановление.** Не забывайте о важности отдыха. Мышцам нужно время для восстановления после тренировок, поэтому не забывайте про дни отдыха.

7. **Отслеживание прогресса.** Ведите дневник тренировок и записывайте свои результаты. Это поможет вам видеть прогресс и мотивировать на дальнейшие достижения.

8. **Поддержка и мотивация.** Найдите единомышленников или присоединитесь к группе по интересам. Поддержка и мотивация от других людей могут стать дополнительным стимулом для достижения целей.

Помните, что похудение — это процесс, который требует времени и терпения. Главное — быть последовательным и не сдаваться при первых трудностях. Удачи вам!

После использования переписку и ассистента можно удалить.

In [60]:
thread.delete()
assistant.delete()

## Добавляем Function Calling

Наш агент может вести переписку и давать советы по фитнесу. Добавим к нему полезную функцию - ведения дневника упражнений. Для этого агенту нужно дать возможность писать некоторую информацию в специальную базу данных. Для простоты будем использовать просто список объектов в памяти, хотя на практике, конечно, имеет смысл использвать какую-нибудь СУБД.

Информация о каждом упражнении будем хранить в виде такого объекта:

In [61]:
from pydantic import BaseModel, Field
from typing import Optional
from datetime import datetime

class Exercise(BaseModel):
    """Эта функция позволяет добавлять информацию о сделанном в зале упражнении."""

    тип: str = Field(description="Тип упражнения (кардио или силовое)", default=None)
    название: str = Field(description="Название упражения", default=None)
    болевые_ощущения : str = Field(description="Болевые ощущения при выполнении упражнения", default=None)
    пульс : int = Field(description="Пульс в момент выполнения упражнения", default=None)
    подходы : int = Field(description="Количество подходов", default=None)
    повторения : int = Field(description="Количество повторений", default=None)

Добавим к нашему боту возможность вызова функции. Для этого при создании ассистента в списке `tools` укажем соответствующий инструмент на основе описанного нами класса - это приведёт к тому, что каждый раз при вызове модели ей будет передаваться подсказка, что ей доступна функция с указанным описанием. 

> **ВАЖНО**: Описание семантики функции берётся из doc-строки в классе `Exercise`, а описания параметров функции - из соответствующих полей `description` каждого поля. Поэтому в классе `Exercise` очень важно подробно документировать все поля.

Также, для надёжности, мы опишем критерии вызова функции в системном промпте.

In [62]:
add_exercise_tool = sdk.tools.function(Exercise)

assistant = create_assistant(model, tools=[add_exercise_tool])
thread = create_thread()

instruction = """
Ты - опытный фитнес-тренер, задача которого - помочь мне тренироваться в зале. Ты можешь
советовать упражнения, давать рекомендации по питанию и т.д. Ты также можешь вести 
дневник выполненных пользователем упражнений - для этого используй функцию `Exercise`.
"""

_ = assistant.update(instruction=instruction)

In [63]:
thread.write("Привет! Я сделал 10 приседаний с отягощением.")
run = assistant.run(thread)
res = run.wait()
res

RunResult(status=<RunStatus.TOOL_CALLS: 5>, error=None, tool_calls=ToolCallList(ToolCall(function=FunctionCall(name='Exercise', arguments={'тип': 'силовое', 'повторения': 10.0, 'название': 'Приседания с отягощением'})),), _message=None, usage=Usage(input_text_tokens=286, completion_tokens=28, total_tokens=314))

Видим, что модель хочеть совершить вызов функции, поэтому возвращает результат со статусом `TOOL_CALLS`, который содержит в себе имя функции и всё необходимые аргументы. Реализовать сам вызов функции - это наша задача:

In [64]:
exercise_db = {}

def add_exercise(thread, exercise):
    if thread.id not in exercise_db:
        exercise_db[thread.id] = []
    exercise_db[thread.id].append(exercise)

In [65]:
import time

thread = create_thread()

thread.write("Привет! Я сделал 10 приседаний с отягощением.")

run = assistant.run(thread)
res = run.wait()
if res.tool_calls:
    result = []
    for f in res.tool_calls:
        print(f" + Вызываю функцию {f.function.name}")
        x = Exercise.model_validate(f.function.arguments)
        x = add_exercise(thread,x)
        result.append({"name": f.function.name, "content": x})
    run.submit_tool_results(result)
    res = run.wait()

res

 + Вызываю функцию Exercise


RunResult(status=<RunStatus.COMPLETED: 4>, error=None, tool_calls=None, _message=Message(id='fvtesmmkh69750apumdt', parts=('Отлично, приседания с отягощением — отличное упражнение для развития силы и массы мышц ног! Если у тебя есть ещё какие-то упражнения или вопросы, не стесняйся спрашивать!',), thread_id='fvtgb5tfi3k7ggj3kr6j', created_by='ajej20rll4tifkelclga', created_at=datetime.datetime(2025, 8, 7, 17, 32, 28, 330239), labels=None, author=Author(id='fvto9tie1meb4q9sg44s', role='ASSISTANT'), citations=(), status=<MessageStatus.COMPLETED: 1>), usage=Usage(input_text_tokens=611, completion_tokens=65, total_tokens=676))

Таким образом, для вызова функции необходимо:
* Сообщить модели о доступных функциях
* При вызове модели обработать функциональный вызов, если в результате вызова модель вернула результат со статусом `TOOL_CALLS`

Для реализации полноценного ассистента нам потребуется ещё добавить функцию распечатки дневника занятий. Поэтому немного структурируем наш код:
* Добавим функцию для обработки вызова прямо в класс с описанием данных
* Создадим класс `Assistant`, который будет реализовывать функциональный вызов

In [66]:
class Exercise(BaseModel):
    """Эта функция позволяет добавлять информацию о сделанном в зале упражнении."""

    тип: str = Field(description="Тип упражнения (кардио или силовое)", default=None)
    название: str = Field(description="Название упражения", default=None)
    болевые_ощущения : str = Field(description="Болевые ощущения при выполнении упражнения", default=None)
    пульс : int = Field(description="Пульс в момент выполнения упражнения", default=None)
    подходы : int = Field(description="Количество подходов", default=None)
    повторения : int = Field(description="Количество повторений", default=None)

    def process(self, thread):
        if thread.id not in exercise_db:
            exercise_db[thread.id] = []
        exercise_db[thread.id].append(self)

class ListExercises(BaseModel):
    """Эта функция позволяет получить список сделанных упражнений"""

    def process(self,thread):
        if thread.id not in exercise_db:
            return "Упражнений нет"
        else:
            return '\n'.join([
                f"{i+1}. {x.название} ({x.тип})" for i,x in enumerate(exercise_db[thread.id])
            ])


In [87]:
class Assistant():
    def __init__(self, tools, instruction, search_tool = None):
        self.tool_map = { x.__name__ : x for x in tools }
        tools = [ sdk.tools.function(x) for x in tools ]
        if search_tool:
            tools.append(search_tool)
        self.assistant = create_assistant(model,tools)
        self.assistant.update(instruction=instruction)

    def __call__(self, thread):
        run = self.assistant.run(thread)
        res = run.wait()
        if res.tool_calls:
            result = []
            for f in res.tool_calls:
                print(f" + Вызываю функцию {f.function.name}, args = {f.function.arguments}")
                if f.function.name in self.tool_map:
                    fn = self.tool_map[f.function.name]
                    obj = fn(**f.function.arguments)
                    x = obj.process(thread)
                    result.append({"name": f.function.name, "content": x})
            run.submit_tool_results(result)
            res = run.wait()
        return res

In [68]:
instruction = """
Ты - опытный фитнес-тренер, задача которого - помочь мне тренироваться в зале. Ты можешь
советовать упражнения, давать рекомендации по питанию и т.д. Ты также можешь вести 
дневник выполненных пользователем упражнений - для этого используй функцию `Exercise`. Чтобы
показать список выполненных упражнений - используй `ListExercises`.
"""

assistant = Assistant([Exercise, ListExercises], instruction)

thread = create_thread()
thread.write("Привет! Я сделал 10 отжиманий от пола!")
print(assistant(thread).text)

 + Вызываю функцию Exercise, args = {'тип': 'силовое', 'повторения': 10.0, 'название': 'отжимания от пола'}
Отлично! Ты выполнил 10 отжиманий от пола. Продолжай в том же духе! Если у тебя есть еще какие-то упражнения или вопросы, не стесняйся спрашивать.


In [69]:
thread.write('А какие упражнения я сегодня делал?')
print(assistant(thread).text)

 + Вызываю функцию ListExercises, args = {}
Сегодня ты сделал 10 отжиманий от пола. Продолжай в том же духе! Если у тебя есть еще какие-то упражнения или вопросы, не стесняйся спрашивать.


In [70]:
exercise_db

{'fvtgb5tfi3k7ggj3kr6j': [Exercise(тип=None, название='Приседания с отягощением', болевые_ощущения=None, пульс=None, подходы=1, повторения=10)],
 'fvt8cth3tb9m06vsdg4l': [Exercise(тип='силовое', название='отжимания от пола', болевые_ощущения=None, пульс=None, подходы=None, повторения=10)]}

## Добавляем RAG

Наш ассистент сам по себе неплохо отвечает на вопросы по тренировкам, но мы можем сделать его ещё умнее, добавив в него дополнительную текстовую информацию. Для этого мы можем использовать RAG - подход, который позволяет искать релевантную информацию по запросу в текстовых базах знаний.

В качестве текстовой базы знаний возьмём несколько популярных текстов о фитнесе:

In [71]:
from glob import glob
import pandas as pd

def get_token_count(filename):
    with open(filename, "r", encoding="utf8") as f:
        return len(model.tokenize(f.read()))

def get_file_len(filename):
    with open(filename, encoding="utf-8") as f:
        l = len(f.read())
    return l

d = [
    {
        "File": fn,
        "Tokens": get_token_count(fn),
        "Chars": get_file_len(fn),
    }
    for fn in glob("data/text-kb/*.txt")
]

df = pd.DataFrame(d)
df

,File,Tokens,Chars
0,data/text-kb\diary.txt,206,931
1,data/text-kb\how-to-begin.txt,664,3249
2,data/text-kb\program.txt,208,937
3,data/text-kb\qna.txt,1416,7098
4,data/text-kb\what-to-take.txt,205,954


В случае с RAG текстовая база знаний делится на небольшие фрагменты, по которым в ходе запроса осуществляется поиск. Оптимальная длина фрагмента определяется опытным путём, но в среднем разумно придерживаться размера 1000-2000 токенов на фрагмент. В нашем случае длина некоторых файлов превышает 1000 токенов, поэтому мы будем использовать **стратегию чанкования**, для разбиения текстовых файлов на более мелкие фрагементы.

## Загружаем файлы в облако

Чтобы RAG мог осущетвлять поиск по фрагментам файлов, нам необходимо построить индекс, а перед этим - загрузить все файлы в облако.

In [72]:
def upload_file(filename):
    return sdk.files.upload(filename, ttl_days=1, expiration_policy="static")

df["Uploaded"] = df["File"].apply(upload_file)

## Строим индекс

Для индексации файлов можно применять следующие стратегии:
* Поиск по эмбеддингам (векторный поиск) - по всем фрагментам текста вычисляются эмбеддинги, и в процессе поиска осуществляется векторный поиск между эмбеддингом запроса и эмбеддингом фрагмента. Это позволяет осуществлять семантический поиск
* Поиск по ключевым словам
* Гибридный поиск, в котором различными способами объединяются результаты поиска по ключевым словам и векторного поиска.


In [73]:
from yandex_cloud_ml_sdk.search_indexes import (
    StaticIndexChunkingStrategy,
    HybridSearchIndexType,
    ReciprocalRankFusionIndexCombinationStrategy,
)

op = sdk.search_indexes.create_deferred(
    df["Uploaded"],
    index_type=HybridSearchIndexType(
        chunking_strategy=StaticIndexChunkingStrategy(
            max_chunk_size_tokens=1000, chunk_overlap_tokens=100
        ),
        combination_strategy=ReciprocalRankFusionIndexCombinationStrategy(),
    ),
)
index = op.wait()

## Собираем RAG-ассистента

Выше при описании класса `Assistant` мы уже предусмотрели возможность передать дополнительно **инструмент поиска**, поэтому для добавления RAG нам достаточно определить такой инструмент поверх индекса и указать его при создании ассистента. Также важно задать хорошую инструкцию (системный промпт): 

In [74]:
search_tool = sdk.tools.search_index(index)

instruction = """
Ты - опытный фитнес-тренер, задача которого - помочь мне тренироваться в зале. Ты можешь
советовать упражнения, давать рекомендации по питанию и т.д. Отвечай на основе имеющейся
дополнительной информации. Ты также можешь вести 
дневник выполненных пользователем упражнений - для этого используй функцию `Exercise`. Чтобы
показать список выполненных упражнений - используй `ListExercises`.
"""

assistant = Assistant([Exercise, ListExercises], instruction, search_tool=search_tool)

thread = create_thread()
thread.write("Что нужно, чтобы начать заниматься в зале?")

res = assistant(thread)
printx(res.text)

Чтобы начать заниматься в зале, вам необходимо:

1. Оценить состояние своего здоровья и поставить цель тренировок (похудение, набор мышечной массы, поддержание формы и т. д.).
2. Выбрать удобную одежду и обувь, которые не сковывают движения.
3. Рассмотреть возможность приобретения пульсометра для контроля сердечного ритма во время упражнений.
4. Записаться на вводный инструктаж с тренером, который объяснит работу тренажёров и правильную технику выполнения упражнений.
5. Планировать три тренировки в неделю для начала, чтобы постепенно увеличивать нагрузку и давать организму привыкнуть.
6. Подумать о работе с тренером хотя бы на начальном этапе для контроля техники выполнения упражнений и предотвращения травм.
7. Не забывать о разминке перед тренировкой и заминке после неё для снижения риска травматизации.

Посмотрим, из каких источников был получен этот ответ:

In [75]:
def print_citations(result):
    for citation in result.citations:
        for source in citation.sources:
            if source.type != "filechunk":
                continue
            print("------------------------")
            print(source.parts[0])

print_citations(res)

------------------------
### Если я хочу начать заниматься в спортзале, то с чего мне стоит начать? В первую очередь нужно оценить состояние своего здоровья и поставить цель. От этого будет зависеть, в каком направлении вы будете работать: силовые тренировки, пилатес, кроссфит, функциональный тренинг или что-то другое. Выберите удобную одежду, которая не сковывает движения, и кроссовки с амортизацией. Также стоит приобрести пульсометр, чтобы контролировать свой ритм и не перегружать сердце во время упражнений. Для каждого возраста существует свой максимальный диапазон частоты сердечных сокращений. Его можно рассчитать по формуле: (220- свой возраст) х 70%/80%/90%. Максимально допустимая частота для безопасного тренинга – 90%. ### Что мне делать в первый день в зале? При первом посещении вы должны записаться на вводный инструктаж, где тренер объяснит работу тренажеров и правильную технику выполнения упражнений. Он входит в стоимость абонемента. Как правило, в первый день вы только изуча

## Добавляем таблицу добавок

Для серьезных тренировок важны также пищевые добавки. Чтобы добавить информацию о них в нашего ассистента, мы нашли таблицу таких добавок в формате markdown:

In [76]:
with open("data/additives.md", encoding="utf-8") as f:
    additives = f.readlines()
additives_all = "".join(additives)

tokens = len(model.tokenize(additives_all))
print(f"Токенов: {tokens}, {len(additives_all)/tokens} chars/token")

Токенов: 6454, 5.105361016423923 chars/token


In [77]:
printx(additives_all[:1040])

| Название добавки                                 | Категория                                      | Назначение                                                                                                 | Доза                          | Рейтинг |
|--------------------------------------------------|-----------------------------------------------|-----------------------------------------------------------------------------------------------------------|-------------------------------|---------|
| Креатин                                          | Для наращивания мышечной массы, силы и ускорения восстановления | Увеличивает физическую силу, ускоряет восстановление; увеличивает массу                                   | 2-20 г в день                 | *****   |
| Глютамин                                         | Для наращивания мышечной массы, силы и ускорения восстановления | Предотвращает распад мышечной ткани; укрепляет иммунную систему                                           | 5-20 г в день                 | *****   

Видим, что табличка большая, поэтому её придётся *чанковать*. Но при этом важно чанковать табличку так, чтобы в каждом фрагмента оставался заголовок таблицы, который определяет семантику столбцов.

Отделим заголовок таблицы:

In [78]:
header = additives[:2]
header

['| Название добавки                                 | Категория                                      | Назначение                                                                                                 | Доза                          | Рейтинг |\n',
 '|--------------------------------------------------|-----------------------------------------------|-----------------------------------------------------------------------------------------------------------|-------------------------------|---------|\n']

Ниже будем чанковать табличку вручную, задав размер чанка в символах для простоты. Мы будем сразу загружать получившиеся фрагменты в облако, минуя диск:

In [79]:
chunk_size = 600 * 5  # около 600 tokens * 5 char/token

s = header.copy()
uploaded_additives = []
for x in additives[2:]:
    s.append(x)
    if len("".join(s)) > chunk_size:
        id = sdk.files.upload_bytes(
            "".join(s).encode(), ttl_days=5, expiration_policy="static",
            mime_type="text/markdown",
        )
        #printx("".join(s))
        uploaded_additives.append(id)
        s = header.copy()
print(f"Uploaded {len(uploaded_additives)} table chunks")

Uploaded 12 table chunks


Теперь добавим эти фрагменты в индекс:

In [80]:
op = index.add_files_deferred(uploaded_additives)
xfiles = op.wait()

Посмотрим, как система стала отвечать на вопросы о добавках:

In [81]:
thread = create_thread()

thread.write("Какие добавки помогают нарастить мышечную массу?")
result = assistant(thread)

printx(result.text)
print_citations(result)

Для наращивания мышечной массы могут помочь следующие добавки:

1. Креатин — увеличивает физическую силу, ускоряет восстановление и способствует увеличению мышечной массы.
2. Глютамин — предотвращает распад мышечной ткани и укрепляет иммунную систему.
3. Аминокислоты с разветвлёнными цепями — предотвращают распад мышечной ткани и притупляют чувство усталости.
4. ZMA (цинк-монометионин-аспартат, магний-аспартат, В6) — повышает уровень тестостерона и ИГФ-1, увеличивает физическую силу и улучшает сон/восстановление.
5. В-гидрокси-В-метилбутират (НМВ) — увеличивает мышечную массу тела и предотвращает распад мышечного белка.

------------------------
| Название добавки                                 | Категория                                      | Назначение                                                                                                 | Доза                          | Рейтинг |
|--------------------------------------------------|-----------------------------------------------|-----------------------------------------------------------------------------------------------------------|-------------------------------|---------|
| Глюкоманнан                                      | Для наращивания мышечной массы, силы и ускорения восстановления | Гелеобразующие волокна; пробиотик, укрепляющий иммунную систему; понижает уровень "плохого" холестерина; понижает уровень сахара в крови | 3-9 г в день                  | ****    |
| Экстракт гриба "шиитейк"                         | Для наращивания мышечной массы, силы и ускорения восстановления | Укрепляет иммунную систему                          

## Изменяем стратегию вызова поиска

В примере выше поисковый инструмент вызывался каждый раз в ответ на каждую реплику пользователя - этим он отличался от рассмотренного ранее механизма вызова инструментов, в котором LLM сама решала, когда сделать вызов функции. Постоянный вызов поиска может быть неэффективным, если большинство реплик отрабатываются с помощью знаний самой LLM.

Инструмент поиска в AI Assistant API позволяет указать стратегию вызова `function`, при которой поисковый инструмент будет вызываться также, как и другие инструменты - по решению LLM. Сделаем ассистента, который содержит в себе информацию о пищевых добавках, а также возможность записи сделанных упражнений.

> Со стратегие вызова поиска как инструмента нам будет сложно интегрировать информацию о фитнес-центре, поскольку сложно ясно сформулировать, когда именно нужно вызывать инструмент поиска. Для добавления общей информации в ассистента лучше использовать поисковый инструмент с постоянным вызовом.

Для начала создадим отдельный индекс для добавок:

In [82]:
op = sdk.search_indexes.create_deferred(
    uploaded_additives,
    index_type=HybridSearchIndexType(
        chunking_strategy=StaticIndexChunkingStrategy(
            max_chunk_size_tokens=1000, chunk_overlap_tokens=100
        ),
        combination_strategy=ReciprocalRankFusionIndexCombinationStrategy(),
    ),
)
additives_index = op.wait()

Теперь определим поисковый инструмент с функциональной стратегией вызова: 

In [112]:
search_tool_additives = sdk.tools.search_index(
    additives_index,
    call_strategy={
        'type': 'function',
        'function': {'name': 'additives', 'instruction': 'call this function whenever you need information on some food additives or medicines one can take to improve fitness results'}
    }
)

Наконец, создаём ассистента:

In [88]:
instruction = """
Ты - опытный фитнес-тренер, задача которого - помочь мне тренироваться в зале. Ты можешь
советовать упражнения, давать рекомендации по питанию и т.д. Ты можешь вести 
дневник выполненных пользователем упражнений - для этого используй функцию `Exercise`. Чтобы
показать список выполненных упражнений - используй `ListExercises`.
"""

assistant = Assistant([Exercise, ListExercises], instruction, search_tool=search_tool_additives)

thread = create_thread()
thread.write("Что нужно, чтобы начать заниматься в зале?")

res = assistant(thread)
print(f"Использовано документов: {len(res.citations)}")
printx(res.text)

Использовано документов: 0


Чтобы начать заниматься в зале, вам потребуется следующее:

1. **Медицинская консультация**: перед началом любых физических упражнений важно убедиться, что ваше здоровье позволяет вам заниматься спортом. Посетите врача и получите рекомендации по тренировкам.

2. **Спортивная одежда и обувь**: выберите удобную одежду, которая не будет сковывать движения, и специализированную обувь для зала, обеспечивающую хорошую амортизацию и поддержку стопы.

3. **План тренировок**: разработайте план тренировок с учетом ваших целей (похудение, набор мышечной массы, улучшение физической формы и т.д.) и уровня подготовки. Лучше всего обратиться к профессиональному тренеру, который поможет составить индивидуальный план.

4. **Базовые знания о технике выполнения упражнений**: правильная техника выполнения упражнений важна для предотвращения травм. Если вы новичок, рекомендуется начать с занятий с тренером, который научит вас правильной технике.

5. **Гидратация и питание**: обеспечьте достаточное потребление воды и сбалансированное питание, чтобы поддерживать энергию во время тренировок и восстановление после них.

6. **Мотивация и регулярность**: регулярные тренировки — ключ к успеху. Найдите источник мотивации, который будет помогать вам придерживаться графика занятий.

7. **Дневник тренировок**: ведите дневник, в котором будете фиксировать свои достижения, прогресс и ощущения от тренировок. Это поможет вам отслеживать свой путь и вносить необходимые коррективы в план тренировок.

In [89]:
thread.write("Какие добавки принимать для увеличения мышечной массы?")

res = assistant(thread)
print(f"Использовано документов: {len(res.citations)}")
printx(res.text)

 + Вызываю функцию additives, args = {'searchQuery': 'добавки для увеличения мышечной массы'}
Использовано документов: 5


Для увеличения мышечной массы часто используются следующие пищевые добавки:

1. **Креатин** — увеличивает физическую силу, ускоряет восстановление и способствует увеличению мышечной массы. Рекомендуемая доза: 2-20 г в день.
2. **Глютамин** — предотвращает распад мышечной ткани и укрепляет иммунную систему. Рекомендуемая доза: 5-20 г в день.
3. **Аминокислоты с разветвленными цепями (ВСАА)** — предотвращают распад мышечной ткани и притупляют чувство усталости. Рекомендуемая доза: 5-10 г в день.
4. **ZMA** — повышает уровень тестостерона и ИГФ-1, увеличивает физическую силу и улучшает сон/восстановление. Рекомендуемая доза: 2-3 капсулы в день.
5. **В-гидрокси-В-метилбутират (НМВ)** — увеличивает мышечную массу тела и предотвращает распад мышечного белка. Рекомендуемая доза: 3-5 г в день.

## Удаляем лишнее

В заключение удалим созданные ресурсы!

> **ВНИМАНИЕ**: Не выполняйте этот код, если у вас есть другие проекты с ассистентами в облаке! Он удаляет все индексы, ассистентов, файлы и переписки! 

In [47]:
for thread in sdk.threads.list():
    print(f" + deleting thread id={thread.id}",end="")
    try:
        thread.delete()
    except:
        print(" ! Error",end="")
    print()
        
for assistant in sdk.assistants.list():
    print(f" + deleting assistant id={assistant.id}")
    assistant.delete()

 + deleting thread id=fvtiinvhmmkgke8mjd7h
 + deleting thread id=fvtun3vd77bme790696h
 + deleting thread id=fvtl8v29c3308dgeu5re
 + deleting thread id=fvtv1s8ilejs0j8u2vkf
 + deleting thread id=fvt78dit8d80e09iidfk
 + deleting thread id=fvt5dbdnnpebsgasimer
 + deleting thread id=fvt3a7b4v5bju9dehp8k
 + deleting thread id=fvt6sj220if7o6ntas5q
 + deleting thread id=fvtice1ne174h5u0vd66
 + deleting thread id=fvt9utcfh382b3n2j4pp
 + deleting thread id=fvtfp9gaf0bv0omgdbgo
 + deleting thread id=fvt8b6nlusdt1j2bhi2o
 + deleting thread id=fvtq733vf8utvqett9t6
 + deleting thread id=fvtl7ce2bnvcsgajdtrf
 + deleting thread id=fvtbd4k1csg2hviq0l6q
 + deleting thread id=fvt0jq1hkfoki6sr6s62
 + deleting thread id=fvtb7unriq6944koonus
 + deleting thread id=fvt9a74fsprjq29boc5f
 + deleting assistant id=fvt6cipttbtqkckn3hoa
 + deleting assistant id=fvt1j5rkg30mgaj9ioog
 + deleting assistant id=fvtjfeot10qvbai02orh
 + deleting assistant id=fvtjtri887d3mdcsvsla
 + deleting assistant id=fvtbk3rheoktemftr

In [49]:
from tqdm.auto import tqdm

for index in sdk.search_indexes.list():
    print(f" + deleting index id={index.id}")
    index.delete()
    
print(" + Deleting files")
for file in tqdm(sdk.files.list()):
    # print(f" + deleting file id={file.id}")
    file.delete()

 + Deleting files


0it [00:00, ?it/s]

0it [00:01, ?it/s]
